In [2]:
! pip install requests

In [3]:
! pip install beautifulsoup4

In [14]:
from bs4 import BeautifulSoup
import requests

page_link = 'http://www.espn.com/nfl/story/_/id/25824920/angry-new-orleans-saints-fans-file-lawsuits-put-billboards'

page_response = requests.get(page_link, timeout = 5)

page_content = BeautifulSoup(page_response.content, "html.parser")

textContent = []
for i in range(0, 26):
    paragraphs = page_content.find_all("p")[i].text
    textContent.append(paragraphs)
    
for i in textContent:
    print (i)

Adam Schefter breaks down Saints owner Gayle Benson's comments about the controversial missed pass interference call in the loss to the Rams. (0:58)
METAIRIE, La. -- New Orleans Saints fans are having their say.
After a stunning missed call by officials helped to keep the Saints out of the Super Bowl, angry fans have been reacting in various ways -- from lawsuits and petitions, to billboards in Atlanta, to boycotts throughout the New Orleans area, to a letter from Louisiana's governor that chastised NFL commissioner Roger Goodell.
In the wake of the no-call that helped decide Sunday's NFC Championship Game, the NFL will discuss whether to make pass-interference penalties reviewable, a source confirmed to ESPN.
Sunday's no-call in the NFC championship game shows us why it's time for the NFL to make pass interference a reviewable play.
Sean Payton said the NFL's head of officials told him the officials "blew the call" by not penalizing the Los Angeles Rams for pass interference late in t

In [ ]:
! pip install -U spacy

In [ ]:
import spacy